veltiCRYS
==========

This notebook runs an example of geometric optimization on a crystal structure using modules of veltiCRYS. It performs energy calculations and approximate local minimization of the energy potential function Buckingham-Coulomb using first derivatives. The Buckingham and Coulomb energy potentials have been expanded using the [Ewald summation] (https://en.wikipedia.org/wiki/Ewald_summation). Both the function and its gradients are custom implementations. 

#### Available energy potential functions and gradients:
- [Coulomb](cysrc/coulomb.pxd)
- [Buckingham](cysrc/buckingham.pxd)

#### Available minimization algorithms:
- [Gradient Descent](https://en.wikipedia.org/wiki/Gradient_descent) as GD
- [Conjugate Gradient](https://en.wikipedia.org/wiki/Nonlinear_conjugate_gradient_method) as CG

The respective methods can be found in [direction.py](pysrc/direction.py).

#### Available step size adaptations (and `method_name`):
- constant step (`steady_step`)
- bisect (`scheduled_bisection`)
- expo (`scheduled_exp`)
- gbisect (`gnorm_scheduled_bisection`)

Other options include:
1. Pausing after every optimization iteration with `usr_flag`.
   If `usr_flag=True` it forces the execution to wait for the user's input to continue. If the input is
    - "no", execution stops
    - "more", execution runs without other pauses
    - anything else, another iteration is perfomred and execution pauses
2. Selecting initial step value with `max_step`.
3. Selecting the frequency of .cif output files with `out`.
4. Forcing the algorithm to reset the direction update to GD with `reset`.
5. Debugging the  gradient with "debug".

Since the energy and gradient calculations are performed with Cython, some functions and classes are implemented using C language to enhance performance. For this reason, the next line needs to be executed in order to compile the code written in C and produce modules compatible with Python:

In [ ]:
!python setup.py build_ext --inplace

In [1]:
import numpy as np
from ase import *
from ase.visualize import view
from ase.io import read as aread

from pysrc.utils import *
from pysrc.direction import *
from IPython.display import HTML

&nbsp;
Some useful functions for input-output and visualizations are provided from the [ASE](https://wiki.fysik.dtu.dk/ase/). *pysrc* and *cyrsc* folders include modules from our custom implementation. The charge_dict dictionary needs to inlcude any elements found in the input structure along with their charge values so that `dict={'elem_name': charge_value}`.
&nbsp;

In [2]:
charge_dict = {
    'O' : -2.,
    'Sr':  2.,
    'Ti':  4.,
    'Cl': -1.,
    'Na':  1.,
    'S' : -2.,
    'Zn':  2.
}

#### Input

The input structures are Atoms (ASE) instances. These can be defined in a Python script or be read from a [CIF](https://en.wikipedia.org/wiki/Crystallographic_Information_File) file. The necessary parameters for the software to work are the ion positions and lattice vectors, which are provided as members of ASE's Atoms class. The Buckingham potential also needs the values for the pairwise parameters A,C,ρ which are emperically determined in literature. These have to be defined in a library file (here *buck.lib*) in the following format:
```
buck
element_name_1a core element_name_1b core  A   ρ C min_dist max_dist
element_name_2a core element_name_2b core  A   ρ C min_dist max_dist
```

In [3]:
datafile = "data/1.cif"
libpath = "libraries"

(folder, structure, atoms) = get_input(datafile)
params = initialise(atoms)
atoms_html = atoms_to_html(atoms)

coul_libfile  = libpath+"/madelung.lib"
buck_libfile  = libpath+"/buck.lib"
buck_libfile2 = libpath+"/radii.lib"

# avoid truncating too many terms
assert((-np.log(params['accuracy'])/params['N']**(1/6)) >= 1)

Using file as input.


In [4]:
HTML(atoms_html)

#### Coulomb energy
_________________________

```python
  Cpot = Coulomb(chemical_symbols, N, charge_dict, alpha, filename)

```

Arguments of this function include:
 
| Argument | Function | 
| ---------------- | -------------------------------------------- |
| N                | Number of ions in unit cell                  |
| chemical_symbols | Ions' chemical symbols in resp. positions    |
| charge_dict      | Dictionary of ion charge per element         |
| filename         | File with Madelung constants (optional)      |
| alpha            | Balance between reciprocal and real space (optional)  |

The cutoff parameters need to be defined using method

```python
  Coulomb.set_cutoff_parameters(self, vects, N, accuracy, real, reciprocal)

```
before each energy calculation, if the unit cell undergoes any changes. The cutoff values are then used to calculate pairwise distances of ions in neighbouring cells using the *inflated_cell_truncation method* in [cutoff.pyx](cysrc/cutoff.pyx).

In [5]:
from cysrc.coulomb import *

Cpot = Coulomb(
    chemical_symbols = params['chemical_symbols'],
    N                = params['N'],
    charge_dict      = charge_dict,
    filename         = coul_libfile
)

Cpot.set_cutoff_parameters(
    vects = params['vects'], 
    N     = params['N'])

Cpot.calc(atoms)
coulomb_energies = Cpot.get_energies()

print('Electrostatic energy')
prettyprint(coulomb_energies)

Electrostatic energy

 Real
-96.64105987560065

 Self
-317.38635089896223

 Reciprocal
100.8327933903345

 All
-313.19461738422837
--------------------------------------------------------------------------------


#### Buckingham energy
_________________________

```python 
  Bpot = Buckingham(filename, chemical_symbols, alpha, radius_lib, radii, limit)

```

Arguments of this function include:

| Argument | Function | 
| ---------------- | -----------------------------------------    |
| filename        | Library file with Buckingham constants        |
| chemical_symbols| Ions' chemical symbols in resp. positions     |
| radius_lib      | Library file with radius value per element ion|
| radii           | Array with radius per ion position (optional) |
| limit           | Lower bound limit of pairwise distances       |
| alpha           | Balance between reciprocal and real space (optional)   |

The cutoff parameters need to be defined using method

```python
  Buckingham.set_cutoff_parameters(self, vects, N, accuracy, real, reciprocal)

```
before each energy calculation, if the unit cell undergoes any changes. The cutoff values are then used to calculate pairwise distances of ions in neighbouring cells using the *inflated_cell_truncation method* in [cutoff.pyx](cysrc/cutoff.pyx).

In [6]:
from cysrc.buckingham import *    

Bpot = Buckingham(
    filename         = buck_libfile, 
    chemical_symbols = params['chemical_symbols'], 
    radius_lib       = buck_libfile2,
    )

Bpot.set_cutoff_parameters(
    vects = params['vects'], 
    N     = params['N'])

Bpot.calc(atoms)
buckingham_energies = Bpot.get_energies()

print('Interatomic energy')
prettyprint(buckingham_energies)

Interatomic energy

 Real
99.75459750131738

 Self
-2.377285695659444

 Reciprocal
-0.4658000498732826

 All
96.91151175578466
--------------------------------------------------------------------------------


#### Descent
_________________________

This is a class that instantiates the optimization run. It can be configured with various tolerances that will make up the stopping criteria of the minimization. Here, the used gradient norm tolerance (`gtol`) used is 0.001. An evaluation of the energy of the initial configuration (`init_energy`) is necessary for the optimization to start. The method that executes the optimisation is

```python
  repeat(self, init_energy, atoms, potentials, outdir, outfile,
    step_func, direction_func, strains, usr_flag, max_step, out)

```

which calls the `iter_step` function for each minimization iteration. Every `iter_step` call returns a dictionary with all values related to the current iteration, so that the returning values include the gradient of the current configuration, the new direction vector for the next step, the ion positions' array, the strains tensor, the lattice vectors' array, the iteration number, the step size used, the gradient norm of the current configuration and the energy value of the current configuration.

Arguments of the repeat function include:

| Argument | Function | 
| ---------------- | -----------------------------------------    |
| init_energy      | Initial energy value                         |
| atoms            | Object of ase Atoms class                    |
| potentials       | Dict. with potential objects indexed by name |
| outdir           | Folder containing any output files           |
| outfile          | Name for output files                        |
| step_func        | Function for line search (optional)          |
| direction_func   | Function for calculating direction vector    |
| strains          | Initial strain array (optional)              |
| usr_flag         | Initiative to stop for input after each iteration (optional)|
| max_step         | Upper bound for step size   (optional)       |
| out              | Interval of output production (optional)     |


In [7]:
from descent import *

potentials     = {}
initial_energy = 0
desc           = Descent(iterno=10000)

initial_energy += coulomb_energies['All']
potentials['Coulomb'] = Cpot

initial_energy += buckingham_energies['All']
potentials['Buckingham'] = Bpot

if not os.path.isdir("output"):
    os.mkdir("output")    

prettyprint({'Chemical Symbols':atoms.get_chemical_symbols(), 'Positions':atoms.positions, \
    'Cell':atoms.get_cell(), 'Total energy':initial_energy})


 Chemical Symbols
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Sr', 'Sr', 'Sr', 'Ti', 'Ti', 'Ti']

 Positions
[[2. 2. 6.]
 [0. 6. 8.]
 [2. 4. 8.]
 [2. 2. 0.]
 [2. 4. 2.]
 [0. 4. 2.]
 [0. 2. 6.]
 [2. 2. 4.]
 [2. 4. 4.]
 [1. 7. 5.]
 [1. 1. 9.]
 [1. 5. 9.]
 [3. 7. 7.]
 [3. 3. 1.]
 [1. 1. 5.]]

 Cell
Cell([4.0, 8.0, 10.0])
------------------------Total energy -216.2831056284437 ------------------------


In [ ]:
from pysrc.linmin import *

iteration = {'Energy': initial_energy}

desc.iterno = 50000
evals, iteration = desc.repeat(
    init_energy=iteration['Energy'],
    atoms=atoms, 
    potentials=potentials, 
    outdir="output",
    outfile=folder+structure,
    direction_func=GD,
    step_func=scheduled_exp,
    usr_flag=False,
    max_step=0.001,
    out=100,
    reset=True,
    debug=False
)

print("Total energy evaluations: {}".format(evals))